In [ ]:
'''
submit of only B3 B4 & B5 models
'''

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# size 384 : All Preds from EB
#pred_b2 = pd.read_csv('../input/rcsiimpreds/sub_EfficientNetB2_384.csv') -- it is decreasing LB score
pred_b3 = pd.read_csv('../input/rcsiimpreds/sub_EfficientNetB3_384_9460.csv') 
pred_b4 = pd.read_csv('../input/rcsiimpreds/sub_EfficientNetB4_384_9498.csv') 
pred_b5 = pd.read_csv('../input/rcsiimpreds/sub_EfficientNetB5_384_9454.csv') 
pred_b6 = pd.read_csv('../input/rcsiimpreds/sub_EfficientNetB6_384_9481.csv')
# class weight
pred_cw_b4 = pd.read_csv('../input/rcsiimpreds/sub_EfficientNetB4_CW_384_9457.csv')
pred_cw_b4.rename(columns = {'target':'target_cw_b4'}, inplace = True)

In [ ]:
# Prediction from B5 run one by one for each fold
pred_512_B6 = pd.read_csv('../input/rcsiimpreds/sub_B5_512_3fold_9466.csv')
pred_512_B6.rename(columns = {'target':'target_B6_512'}, inplace = True)

In [ ]:
pred_tta_b3 = pd.read_csv('../input/rcsiimpreds/siim_tta_b3_9458.csv')
pred_tta_b3.rename(columns = {'target':'target_tta_b3'}, inplace = True)

pred_tta_b4 = pd.read_csv('../input/rcsiimpreds/siim_tta_b4_9473.csv')
pred_tta_b4.rename(columns = {'target':'target_tta_b4'}, inplace = True)

result_tta = pd.merge(pred_b3, pred_b4, on='image_name',suffixes=('_tta_b3','_tta_b4'))

result_tta.head()

In [ ]:
# B4 on 512
#pred_512_b4   = pd.read_csv('../input/rcsiimpreds/sub_EfficientNetB4_512_9448.csv')

In [ ]:
#pred_512_B5_F3.rename(columns = {'target':'target_F3'}, inplace = True) 

In [ ]:
result1 = pd.merge(pred_b3, pred_b4, on='image_name',suffixes=('_b3','_b4'))

In [ ]:
result1.head()

In [ ]:
result2 = pd.merge(pred_b5, pred_b6, on='image_name',suffixes=('_b5','_b6'))

In [ ]:
result2.head()

In [ ]:
semi_final = pd.merge(result1, result2, on='image_name')

In [ ]:
semi_final.head()

In [ ]:
result3 = pd.merge(pred_cw_b4, pred_512_B6, on='image_name',suffixes=('_cw_b4','_B6_512'))
result3.head()

In [ ]:
final = pd.merge(semi_final, result3, on='image_name')
final.head()

In [ ]:
final = pd.merge(final, result_tta, on='image_name')
final.head()

In [ ]:
# Adding Karan's model prediction
pred_kr_b3 = pd.read_csv('../input/rcsiimpreds/KR_sub_EfficientNetB3_512_9520.csv')
pred_kr_b3.rename(columns = {'target':'target_kr_b3'}, inplace = True)

pred_kr_b4 = pd.read_csv('../input/rcsiimpreds/KR_sub_EfficientNetB4_512_9499.csv')
pred_kr_b4.rename(columns = {'target':'target_kr_b4'}, inplace = True)

pred_kr_eb3 = pd.read_csv('../input/rcsiimpreds/KR_sub_eb3_512_9554.csv')
pred_kr_eb3.rename(columns = {'target':'target_kr_eb3'}, inplace = True)

#pred_cw_b4 = pd.read_csv('')
#pred_cw_b4.rename(columns = {'target':'target_cw_b4'}, inplace = True)


In [ ]:
kr_result = pd.merge(pred_kr_b3, pred_kr_b4, on='image_name',suffixes=('_b3','_b4'))

kr_result = pd.merge(kr_result, pred_kr_eb3, on='image_name',suffixes=('_b3','_b4'))
kr_result.head()

In [ ]:
final = pd.merge(final, kr_result, on='image_name')
final.head()

In [ ]:
pred_256_b4 = pd.read_csv('../input/rcsiimpreds/sub_EfficientNetB4_256_9496.csv')
pred_256_b4.rename(columns = {'target':'target_256_b4'}, inplace = True)
pred_256_b4.head()

In [ ]:
final = pd.merge(final, pred_256_b4, on='image_name')
final.head()

In [ ]:
# Removing low LB score prediction
# target_tta_b3, target_b5, target_cw_b4
#
final['target'] = ( (final.target_b4)  + 
                   (final.target_b6)  + 
                   final.target_tta_b4 +
                   final.target_kr_b3 + final.target_kr_b4 + final.target_kr_eb3 +
                   final.target_256_b4
                  ) / 7
final.head()

In [ ]:
from scipy.stats import gmean

In [ ]:
# Removing low LB score prediction
# target_tta_b3, target_b5, target_cw_b4
#
target_array =  np.array([final.target_b4,    final.target_b6   ,  final.target_tta_b4, final.target_B6_512,
                       final.target_kr_b3, final.target_kr_b4,  final.target_kr_eb3, final.target_256_b4])
final['target'] = gmean(target_array)
final.head()

In [ ]:
final.head()

In [ ]:
submit_file = final[['image_name', 'target']]

In [ ]:
submit_file.head()

In [ ]:
submit_file.to_csv('submission.csv', index = False)

In [ ]:
submit_file.target.hist()

# Min MAX

import numpy as np
import pandas as pd 
import os 

# list file here to remove
to_remove = ['sub_EfficientNetB2_384.csv',
             'sub_EfficientNetB4_512_9448.csv',
             'sub_EfficientNetB3_512_9403.csv']

def MinMaxBestBaseStacking(input_folder, best_base, output_path):
    sub_base = pd.read_csv(best_base)
    all_files = os.listdir(input_folder)
    
    all_files.remove(to_remove[0])
    all_files.remove(to_remove[1])
    all_files.remove(to_remove[2])
        

    # Read and concatenate submissions
    outs = [pd.read_csv(os.path.join(input_folder, f), index_col=0) for f in all_files]
    concat_sub = pd.concat(outs, axis=1)
    cols = list(map(lambda x: "target" + str(x), range(len(concat_sub.columns))))
    concat_sub.columns = cols
    concat_sub.reset_index(inplace=True)
    print(concat_sub.head())

    # get the data fields ready for stacking
    col_number = len(all_files) + 1
    concat_sub['is_iceberg_max'] = concat_sub.iloc[:, 1:col_number].max(axis=1)
    concat_sub['is_iceberg_min'] = concat_sub.iloc[:, 1:col_number].min(axis=1)
    concat_sub['is_iceberg_mean'] = concat_sub.iloc[:,1:col_number].mean(axis=1)
    concat_sub['is_iceberg_median'] = concat_sub.iloc[:, 1:col_number].median(axis=1)

    # set up cutoff threshold for lower and upper bounds
    cutoff_lo = 0.73
    cutoff_hi = 0.33

    concat_sub['is_iceberg_base'] = sub_base['target']
    concat_sub['target'] = np.where(np.all(concat_sub.iloc[:, 1:col_number] > cutoff_lo, axis=1),
                                        concat_sub['is_iceberg_max'],
                                        np.where(np.all(concat_sub.iloc[:, 1:col_number] < cutoff_hi, axis=1),
                                                 concat_sub['is_iceberg_min'],
                                                 concat_sub['is_iceberg_base']))
    concat_sub[['image_name', 'target']].to_csv(output_path,
                                            index=False, float_format='%.12f')


MinMaxBestBaseStacking('../input/rcsiimpreds/', 
                       '../input/rcsiimpreds/sub_EfficientNetB4_384_9498.csv', 
                       'submission_minmax.csv') # 0.9526 

In [ ]:
#all_files = os.listdir('../input/rcsiimpreds/')
#print(all_files)